In [1]:
# câu1
#a:
class NhanVien:
    def __init__(self, ma_nv, luong_cb):
        self.__ma_nv = ma_nv          # Thuộc tính private
        self.__luong_cb = luong_cb    # Thuộc tính private

    def get_ma_nv(self):
        return self.__ma_nv

    def get_luong_cb(self):
        return self.__luong_cb

    def tinh_luong(self):
        return self.__luong_cb

class NhanVienTangCa(NhanVien):
    def tinh_luong(self):
        # Ghi đè: lương = lương_cb * 1.2
        return self.get_luong_cb() * 1.2

class NhanVienChuyenChuyen(NhanVien):
    def tinh_luong(self):
        # Ghi đè: lương = lương_cb + 500.000
        return self.get_luong_cb() + 500000

# Phần main
if __name__ == "__main__":
    luong_mac_dinh = 10000000.0
    ds_nhan_vien = [
        NhanVien("NV01", luong_mac_dinh),
        NhanVienTangCa("TC01", luong_mac_dinh),
        NhanVienChuyenChuyen("CC01", luong_mac_dinh)
    ]

    print(f"{'Mã NV':<10} | {'Tiền lương':<15}")
    print("-" * 30)
    for nv in ds_nhan_vien:
        print(f"{nv.get_ma_nv():<10} | {nv.tinh_luong():>15,.0f} VNĐ")

Mã NV      | Tiền lương     
------------------------------
NV01       |      10,000,000 VNĐ
TC01       |      12,000,000 VNĐ
CC01       |      10,500,000 VNĐ


In [2]:
#b
import numpy as np

san_luong = [12, None, 8, 15, None, 10]

# 1. Chuyển thành mảng NumPy, ép kiểu float để chứa NaN
arr = np.array(san_luong, dtype=float)

# Reshape thành 2x3
arr_2x3 = arr.reshape(2, 3)

# Thay thế NaN bằng giá trị trung bình (bỏ qua NaN khi tính mean)
mean_val = np.nanmean(arr_2x3)
arr_2x3[np.isnan(arr_2x3)] = mean_val
print("Mảng sau khi xử lý NaN:\n", arr_2x3)

# 2. Trích xuất phần tử > 9 và tìm giá trị nhỏ nhất
gt_lon_hon_9 = arr_2x3[arr_2x3 > 9]
min_val = np.min(gt_lon_hon_9)

print("Các phần tử > 9:", gt_lon_hon_9)
print("Giá trị nhỏ nhất trong tập đó:", min_val)

Mảng sau khi xử lý NaN:
 [[12.   11.25  8.  ]
 [15.   11.25 10.  ]]
Các phần tử > 9: [12.   11.25 15.   11.25 10.  ]
Giá trị nhỏ nhất trong tập đó: 10.0


In [5]:
#câu 2:
import pandas as pd
import sqlite3

#Đọc dữ liệu
df = pd.read_csv('DATA/nhan_su.csv')

# a: Lọc dữ liệu
# Chứa "NS" (không phân biệt hoa thường), xử lý NaN trong Ma_NV bằng na=False
# Nam_Cong_Tac >= 2021
df_filtered = df[
    (df['Ma_NV'].str.contains('NS', case=False, na=False)) & 
    (df['Nam_Cong_Tac'] >= 2021)
].copy()

# b: Làm sạch dữ liệu
# Điền NaN trong Thanh_Tich bằng giá trị trung bình của tập đã lọc
mean_thanh_tich = df_filtered['Thanh_Tich'].mean()
df_filtered['Thanh_Tich'] = df_filtered['Thanh_Tich'].fillna(mean_thanh_tich)

# c: Tổng hợp dữ liệu (Groupby)
summary_pb = df_filtered.groupby('Phong_Ban')['Thanh_Tich'].sum().reset_index()
print("Tổng điểm thành tích theo phòng ban:\n", summary_pb)

# d: Biến đổi bảng (Pivot Table)
pivot_table = df_filtered.pivot_table(
    index='Phong_Ban', 
    columns='Nam_Cong_Tac', 
    values='Thanh_Tich', 
    aggfunc='sum', 
    fill_value=0
)
print("\nBảng xoay chiều (Pivot Table):\n", pivot_table)

# e: Lưu trữ và Truy vấn
# Kết nối SQLite (tạo file HeThong.db)
conn = sqlite3.connect('HeThong.db')

# Lưu kết quả pivot table vào database
pivot_table.to_sql('bang_luong_2024', conn, if_exists='replace')

# Thực hiện truy vấn hiển thị 5 dòng đầu
query = "SELECT * FROM bang_luong_2024 LIMIT 5"
df_check = pd.read_sql(query, conn)
print("\n5 dòng đầu tiên từ SQLite:\n", df_check)

# Đóng kết nối
conn.close()

Tổng điểm thành tích theo phòng ban:
    Phong_Ban  Thanh_Tich
0  KinhDoanh  348.333333
1    KyThuat  268.333333
2     NhanSu  178.333333

Bảng xoay chiều (Pivot Table):
 Nam_Cong_Tac   2021       2022  2023        2024
Phong_Ban                                       
KinhDoanh     165.0  88.333333  95.0    0.000000
KyThuat        88.0   0.000000   0.0  180.333333
NhanSu          0.0  88.333333  90.0    0.000000

5 dòng đầu tiên từ SQLite:
    Phong_Ban   2021       2022  2023        2024
0  KinhDoanh  165.0  88.333333  95.0    0.000000
1    KyThuat   88.0   0.000000   0.0  180.333333
2     NhanSu    0.0  88.333333  90.0    0.000000
